In [ ]:
# ==============================================================================
# PASO 1: Instalar todas las librerías necesarias
# ==============================================================================
print("--- Instalando y actualizando librerías ---")
import os
import subprocess
import time
import re
import urllib.request

# Usamos subprocess.run para asegurarnos de que los comandos se ejecuten correctamente
# e instalamos las librerías de forma silenciosa (-q).
# Forzamos la reinstalación de yt-dlp a la última versión para evitar errores de descarga.
subprocess.run([
    "pip", "install", "streamlit", "numpy", "scipy", "matplotlib", "pandas",
    "soundfile", "pydub", "browser-cookie3", "-q"
], check=True)
subprocess.run([
    "pip", "install", "--force-reinstall", "https://github.com/yt-dlp/yt-dlp/archive/master.tar.gz", "-q"
], check=True)

--- Instalando y actualizando librerías ---


CompletedProcess(args=['pip', 'install', '--force-reinstall', 'https://github.com/yt-dlp/yt-dlp/archive/master.tar.gz', '-q'], returncode=0)

In [ ]:
# ==============================================================================
# PASO 2: Crear la estructura de carpetas y los archivos de la aplicación
# ==============================================================================
print("\n--- Creando archivos de la aplicación ---")
if not os.path.exists("pages"):
    os.makedirs("pages")


--- Creando archivos de la aplicación ---


In [29]:
# ==============================================================================
# PASO 2: Crear la estructura de carpetas y el archivo principal
# ==============================================================================
%%writefile 0_🏠_Inicio.py
import streamlit as st
import os

st.set_page_config(
    page_title="Parcial 2 - Señales y Sistemas",
    page_icon="🎓",
    layout="wide"
)

st.write("# Parcial 2: Transformada de Laplace 🎓")
st.write("### Señales y Sistemas 2025-I")
st.write("**Realizado por:** Juan Esteban Montero Giraldo")
st.write("**Profesor:** Andrés Marino Álvarez Meza")

st.sidebar.success("Selecciona un problema del parcial.")

st.markdown(
    '''
    Esta aplicación interactiva contiene las soluciones a los problemas del segundo parcial
    de SyS.

    ### Contenido del Parcial:

    1.  **Análisis de Sistema Mecánico y Eléctrico:**
        - Simulación de un sistema masa-resorte-amortiguador y su análogo eléctrico RLC.
        - Visualización de diagramas de Bode, polos y ceros, y respuestas temporales.

    2.  **Modulación y Demodulación SSB-AM:**
        - Simulación del proceso de modulación y demodulación de Amplitud en Banda Lateral Única (SSB).
        - Uso de señales de pulso y audio como mensajes.
        - Implementación y visualización de filtros IIR.

    **👈 Selecciona uno de los problemas en la barra lateral** para ver la solución interactiva.
    '''
)


Overwriting 0_🏠_Inicio.py


In [35]:
# ==============================================================================
# PASO 3: Crear el archivo para el Punto 1
# ==============================================================================
%%writefile pages/1_⚙️_Sistema_Mecánico_Eléctrico.py
import streamlit as st
import numpy as np
import scipy.signal as signal
import matplotlib.pyplot as plt
import pandas as pd

# --- Configuración de la Página ---
st.set_page_config(
    page_title="Análisis de Sistemas de 2º Orden",
    page_icon="⚙️",
    layout="wide",
)

# --- Estilo de Matplotlib ---
plt.style.use('fivethirtyeight')
plt.rcParams['axes.facecolor'] = 'white'
plt.rcParams['figure.facecolor'] = 'white'


# --- Funciones de Ayuda ---

def calculate_temporal_params(t, y_step, final_value):
    '''Calcula los parámetros temporales de la respuesta al escalón.'''
    params = {
        'Tiempo de Levantamiento (tr)': 'N/A',
        'Sobre-impulso Máximo (Mp)': 'N/A',
        'Tiempo de Pico (tp)': 'N/A',
        'Tiempo de Establecimiento (ts)': 'N/A'
    }
    if abs(final_value) < 1e-6: return params
    try:
        t_10_indices = np.where(y_step >= 0.1 * final_value)[0]
        t_90_indices = np.where(y_step >= 0.9 * final_value)[0]
        if t_10_indices.size > 0 and t_90_indices.size > 0:
            params['Tiempo de Levantamiento (tr)'] = f"{t[t_90_indices[0]] - t[t_10_indices[0]]:.3f} s"
    except IndexError: pass
    max_val = np.max(y_step)
    if max_val > final_value:
        params['Sobre-impulso Máximo (Mp)'] = f"{((max_val - final_value) / final_value) * 100:.2f} %"
        params['Tiempo de Pico (tp)'] = f"{t[np.argmax(y_step)]:.3f} s"
    else:
        params['Sobre-impulso Máximo (Mp)'] = "0.00 %"
    try:
        settling_mask = np.abs(y_step - final_value) > 0.02 * abs(final_value)
        if np.any(settling_mask):
            last_out_of_bounds = np.where(settling_mask)[0][-1]
            if last_out_of_bounds + 1 < len(t):
                 params['Tiempo de Establecimiento (ts)'] = f"{t[last_out_of_bounds + 1]:.3f} s"
        else:
            params['Tiempo de Establecimiento (ts)'] = f"{t[0]:.3f} s"
    except IndexError: pass
    return params

# --- UI Principal ---
st.title("🔬 Dashboard de Simulación de Sistemas de 2º Orden")
st.markdown("Este dashboard muestra le equivalencia entre sistemas mecánico-eléctrico, y su desarrollo frente a varios tipos de entrada temporales y varios tipos de respuesta.")

# --- Barra Lateral de Configuración ---
st.sidebar.title("Parámetros del Sistema")

response_type = st.sidebar.selectbox(
    "1. Seleccione el tipo de respuesta:",
    ('Subamortiguada', 'Amortiguamiento Crítico', 'Sobreamortiguada', 'Inestable'),
)

loop_type = st.sidebar.radio(
    "2. Seleccione el tipo de lazo:",
    ('Lazo Abierto', 'Lazo Cerrado'),
    horizontal=True
)

if response_type == 'Subamortiguada':
    zeta = st.sidebar.slider("3. Factor de Amortiguamiento (ζ)", 0.01, 0.99, 0.3, 0.01)
elif response_type == 'Amortiguamiento Crítico':
    zeta = 1.0; st.sidebar.markdown("ζ = 1 (Fijo)")
elif response_type == 'Sobreamortiguada':
    zeta = st.sidebar.slider("3. Factor de Amortiguamiento (ζ)", 1.01, 5.0, 1.5, 0.01)
else: # Inestable
    zeta = st.sidebar.slider("3. Factor de Amortiguamiento (ζ)", -1.0, -0.01, -0.5, 0.01)

wn = st.sidebar.slider("4. Frecuencia Natural (ωn) [rad/s]", 1.0, 20.0, 5.0, 0.1)

# --- Lógica del Sistema ---
num_ol = [wn**2]
den_ol = [1, 2 * zeta * wn, wn**2]
tf_ol = signal.TransferFunction(num_ol, den_ol)

den_cl = np.polyadd(den_ol, num_ol)
tf_cl = signal.TransferFunction(num_ol, den_cl)

if loop_type == 'Lazo Abierto':
    tf_current = tf_ol
    st.header("Análisis del Sistema en Lazo Abierto: $H(s)$")
    st.latex(f"H(s) = \\frac{{{wn**2:.2f}}}{{s^2 + {2*zeta*wn:.2f}s + {wn**2:.2f}}}")
else:
    tf_current = tf_cl
    st.header("Análisis del Sistema en Lazo Cerrado: $H_{{Lazo cerrado}}(s)$")
    st.latex(f"H_{{Lazo cerrado}}(s) = \\frac{{{tf_cl.num[0]:.2f}}}{{s^2 + {tf_cl.den[1]:.2f}s + {tf_cl.den[2]:.2f}}}")

# --- Pestañas de Visualización ---
tab1, tab2, tab3, tab4 = st.tabs([
    "Diagramas de Polos, Ceros y Bode",
    "Respuesta al Impulso",
    "Respuesta al Escalón",
    "Respuesta a la Rampa"
])

is_stable = np.all(np.real(tf_current.poles) < 0)

if is_stable:
    dominant_pole_real = np.max(np.real(tf_current.poles))
    t_final = 5 / abs(dominant_pole_real) if abs(dominant_pole_real) > 1e-3 else 50
else:
    t_final = 15
t_final = min(max(t_final, 1), 50)
t_vec = np.linspace(0, t_final, 2000)

with tab1:
    col1, col2 = st.columns(2)
    with col1:
        fig_pz, ax_pz = plt.subplots()
        poles, zeros = tf_current.poles, tf_current.zeros
        if zeros.size > 0: ax_pz.plot(np.real(zeros), np.imag(zeros), 'o', markersize=10, label='Ceros', color='#007acc')
        ax_pz.plot(np.real(poles), np.imag(poles), 'x', markersize=10, mew=3, label='Polos', color='#d62728')
        ax_pz.grid(True); ax_pz.set_title("Diagrama de Polos y Ceros"); ax_pz.set_xlabel("Eje Real"); ax_pz.set_ylabel("Eje Imaginario")
        ax_pz.axhline(0, color='black', lw=0.5); ax_pz.axvline(0, color='black', lw=0.5)
        ax_pz.legend(fancybox=True, framealpha=1, shadow=True, borderpad=1)
        st.pyplot(fig_pz)
    with col2:
        w, mag, phase = signal.bode(tf_current)
        fig_bode, (ax_mag, ax_phase) = plt.subplots(2, 1, figsize=(6, 6), sharex=True)
        ax_mag.semilogx(w, mag, color='#007acc'); ax_mag.set_ylabel("Magnitud (dB)"); ax_mag.set_title("Diagrama de Bode")
        ax_phase.semilogx(w, phase, color='#ff7f0e'); ax_phase.set_xlabel("Frecuencia (rad/s)"); ax_phase.set_ylabel("Fase (grados)")
        st.pyplot(fig_bode)

with tab2:
    t_imp, y_imp = signal.impulse(tf_current, T=t_vec)
    fig_imp, ax_imp = plt.subplots(figsize=(10, 5))
    ax_imp.plot(t_imp, y_imp, label="Respuesta al Impulso", color='#2ca02c', linewidth=2.5)
    ax_imp.set_title("Respuesta al Impulso", fontsize=14); ax_imp.set_xlabel("Tiempo (s)"); ax_imp.set_ylabel("Amplitud"); ax_imp.legend(); ax_imp.grid(True)
    st.pyplot(fig_imp)
    st.info("Nota: Los parámetros temporales estándar (tr, Mp, tp, ts) se definen para la respuesta al escalón.")

with tab3:
    t_step, y_step = signal.step(tf_current, T=t_vec)
    fig_step, ax_step = plt.subplots(figsize=(10, 5))
    ax_step.plot(t_step, y_step, label="Respuesta al Escalón", color='#9467bd', linewidth=2.5)
    if is_stable and y_step.size > 0:
        ax_step.axhline(y_step[-1], color='gray', linestyle='--', label=f'Valor Final: {y_step[-1]:.2f}')
    ax_step.set_title("Respuesta al Escalón", fontsize=14); ax_step.set_xlabel("Tiempo (s)"); ax_step.set_ylabel("Amplitud"); ax_step.legend(); ax_step.grid(True)
    st.pyplot(fig_step)
    if is_stable and y_step.size > 0:
        st.write("**Parámetros de la Respuesta al Escalón:**")
        params = calculate_temporal_params(t_step, y_step, y_step[-1])
        df_params = pd.DataFrame(params.items(), columns=['Parámetro', 'Valor']); st.table(df_params.set_index('Parámetro'))
    else:
        st.warning("El sistema es inestable. No se calculan parámetros temporales y no hay un valor final de establecimiento.")

with tab4:
    t_ramp, y_ramp, _ = signal.lsim(tf_current, U=t_vec, T=t_vec)
    fig_ramp, ax_ramp = plt.subplots(figsize=(10, 5))
    ax_ramp.plot(t_ramp, y_ramp, label="Respuesta a la Rampa", color='#8c564b', linewidth=2.5)
    ax_ramp.plot(t_ramp, t_ramp, '--', label="Entrada Rampa Ideal", color='gray')
    ax_ramp.set_title("Respuesta a la Rampa", fontsize=14); ax_ramp.set_xlabel("Tiempo (s)"); ax_ramp.set_ylabel("Amplitud"); ax_ramp.legend(); ax_ramp.grid(True)
    st.pyplot(fig_ramp)
    st.info("Nota: Los parámetros temporales estándar (tr, Mp, tp, ts) se definen para la respuesta al escalón.")

st.sidebar.title("Valores Estimados de Componentes")
if response_type == 'Inestable':
    st.sidebar.warning("Los componentes físicos pasivos no pueden ser negativos. No se calculan valores para sistemas inestables.")
else:
    m = 1.0; k = m * wn**2; c = 2 * zeta * wn * m
    st.sidebar.markdown("**Sistema Mecánico (m=1kg):**")
    st.sidebar.markdown(f"- **Rigidez (k):** :red[{k:.2f} N/m]")
    st.sidebar.markdown(f"- **Amortiguador (c):** :red[{c:.2f} Ns/m]")
    st.sidebar.markdown("**Sistema Eléctrico Análogo (C=1000µF):**")
    C_elec = 1000e-6
    if k > 0 and c > 0:
        L_elec = (m/k) / C_elec; R_elec = (L_elec * k) / c
        st.sidebar.markdown(f"- **Inductancia (L):** :red[{L_elec * 1e3:.2f} mH]")
        st.sidebar.markdown(f"- **Resistencia (R):** :red[{R_elec:.2f} Ω]")
    else:
        st.sidebar.markdown(f"- **Componentes:** `No calculables`")

Overwriting pages/1_⚙️_Sistema_Mecánico_Eléctrico.py


In [40]:
# ==============================================================================
# PASO 4: Crear el archivo para el Punto 2
# ==============================================================================
%%writefile pages/2_📡_Modulación_SSB-AM.py
import streamlit as st
import numpy as np
import scipy.signal as signal
from scipy.fft import fft, fftfreq
import matplotlib.pyplot as plt
from scipy.io.wavfile import read, write
from pydub import AudioSegment
import io
import os
import subprocess
import yt_dlp
import browser_cookie3

# --- Configuración de la Página ---
st.set_page_config(
    page_title="Simulador de Modulación SSB-AM",
    page_icon="📡",
    layout="wide",
)
plt.style.use('seaborn-v0_8-darkgrid')

# --- Funciones de Ayuda ---

def plot_signal(t, sig, title, xlabel="Tiempo (s)", y_max=0):
    fig, ax = plt.subplots(figsize=(10, 3))
    ax.plot(t, sig)
    ax.set_title(title, fontsize=14)
    ax.set_xlabel(xlabel)
    ax.set_ylabel("Amplitud")
    if y_max > 0:
        ax.set_ylim([-y_max, y_max])
    ax.grid(True)
    st.pyplot(fig)

def plot_spectrum(sig, fs, title, x_lim=0):
    n = len(sig)
    yf = fft(sig)
    xf = fftfreq(n, 1 / fs)
    fig, ax = plt.subplots(figsize=(10, 3))
    ax.plot(np.fft.fftshift(xf), 2.0/n * np.abs(np.fft.fftshift(yf)))
    ax.set_title(title, fontsize=14)
    ax.set_xlabel("Frecuencia (Hz)")
    ax.set_ylabel("Magnitud")
    if x_lim > 0:
        ax.set_xlim([-x_lim, x_lim])
    ax.grid(True)
    st.pyplot(fig)

def plot_filter(b, a, fs, title):
    w, h = signal.freqz(b, a, worN=8000)
    z, p, k = signal.tf2zpk(b, a)
    fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(12, 4))
    fig.suptitle(title, fontsize=16)
    ax1.plot(0.5 * fs * w / np.pi, 20 * np.log10(abs(h) + np.finfo(float).eps), 'b')
    ax1.set_ylabel('Amplitud [dB]', color='b')
    ax1.set_xlabel('Frecuencia [Hz]')
    ax1.set_title("Diagrama de Bode (Magnitud)")
    ax1.grid(True)
    circulo_unitario = plt.Circle((0,0), 1, fill=False, color='gray', ls='--')
    ax2.add_artist(circulo_unitario)
    ax2.plot(np.real(z), np.imag(z), 'o', markersize=8, fillstyle='none', label='Ceros')
    ax2.plot(np.real(p), np.imag(p), 'x', markersize=8, label='Polos')
    ax2.set_aspect('equal')
    ax2.set_xlim([-1.5, 1.5])
    ax2.set_ylim([-1.5, 1.5])
    ax2.set_title("Plano de Polos y Ceros")
    ax2.set_xlabel("Eje Real")
    ax2.set_ylabel("Eje Imaginario")
    ax2.grid(True)
    ax2.legend(fancybox=True, framealpha=1, shadow=True, borderpad=1)
    st.pyplot(fig)

# --- Función de Descarga de YouTube ---
@st.cache_data
def download_yt_audio(url):
    audio_filename = "downloaded_audio.wav"
    cookies = None
    try:
        cookies = browser_cookie3.firefox()
    except Exception:
        try:
            cookies = browser_cookie3.chrome()
        except Exception:
            st.warning("No se pudieron cargar cookies del navegador. La descarga podría fallar.")

    ydl_opts = {
        'format': 'bestaudio/best', 'outtmpl': 'temp_audio.%(ext)s',
        'postprocessors': [{'key': 'FFmpegExtractAudio', 'preferredcodec': 'wav'}],
        'quiet': True, 'no_warnings': True, 'overwrites': True,
    }
    if cookies: ydl_opts['cookiejar'] = cookies

    try:
        with st.spinner("Descargando audio completo de YouTube..."):
            with yt_dlp.YoutubeDL(ydl_opts) as ydl:
                ydl.download([url])
            if os.path.exists("temp_audio.wav"):
                if os.path.exists(audio_filename): os.remove(audio_filename)
                os.rename("temp_audio.wav", audio_filename)
                return audio_filename
    except Exception as e:
        st.error(f"Error al procesar el enlace de YouTube: {e}")
    return None

# --- UI Principal ---
st.title("📡 Simulador de Modulación y Demodulación SSB-AM")
st.markdown("Este dashboard visualiza el proceso de modulación SSB y su demodulación coherente.")

st.sidebar.title("Configuración")
msg_type = st.sidebar.radio(
    "1. Seleccione la fuente de la señal mensaje:",
    ('Pulso Rectangular', 'Enlace de YouTube', 'Subir Archivo de Audio')
)

fs = 48000
message = None
audio_duration = 5 # Duración del segmento para análisis

if msg_type == 'Pulso Rectangular':
    st.info("Señal de pulso rectangular de 1 segundo generada automáticamente.")
    t = np.linspace(0, audio_duration, audio_duration * fs, endpoint=False)
    message = np.zeros_like(t)
    start_time = 0.2
    end_time = 1.2
    message[int(start_time*fs):int(end_time*fs)] = 1

elif msg_type == 'Enlace de YouTube':
    yt_url = st.text_input("Pegue el enlace del video de YouTube aquí:", "https://www.youtube.com/watch?v=g_p9H2cr31o")
    if yt_url:
        audio_file = download_yt_audio(yt_url)
        if audio_file:
            fs, audio_data = read(audio_file)
            if audio_data.ndim > 1: audio_data = audio_data.mean(axis=1)

            total_duration = len(audio_data) / fs
            max_start = int(total_duration - audio_duration)

            if max_start < 0:
                st.warning("El audio es más corto que 5 segundos. Se usará el audio completo.")
                message = audio_data.astype(float) / np.max(np.abs(audio_data))
            else:
                start_second = st.number_input(
                    "Segundo de inicio para el análisis:",
                    min_value=0,
                    max_value=max_start,
                    value=min(30, max_start),
                    step=1
                )
                start_sample = start_second * fs
                end_sample = start_sample + (audio_duration * fs)
                message_segment = audio_data[start_sample:end_sample]
                message = message_segment.astype(float) / np.max(np.abs(message_segment))

                st.write("**Segmento de 5 segundos para análisis:**")
                st.audio(message, sample_rate=fs)


elif msg_type == 'Subir Archivo de Audio':
    uploaded_file = st.file_uploader("Suba un archivo .wav o .mp3", type=["wav", "mp3"])
    if uploaded_file:
        try:
            if uploaded_file.name.endswith('.mp3'):
                audio = AudioSegment.from_mp3(uploaded_file)
                buf = io.BytesIO()
                audio.export(buf, format="wav")
                fs, audio_data = read(buf)
            else:
                fs, audio_data = read(uploaded_file)
            if audio_data.ndim > 1: audio_data = audio_data.mean(axis=1)

            total_duration = len(audio_data) / fs
            max_start = int(total_duration - audio_duration)

            if max_start < 0:
                st.warning("El audio es más corto que 5 segundos. Se usará el audio completo.")
                message = audio_data.astype(float) / np.max(np.abs(audio_data))
            else:
                start_second = st.number_input(
                    "Segundo de inicio para el análisis:",
                    min_value=0,
                    max_value=max_start,
                    value=0,
                    step=1
                )
                start_sample = start_second * fs
                end_sample = start_sample + (audio_duration * fs)
                message_segment = audio_data[start_sample:end_sample]
                message = message_segment.astype(float) / np.max(np.abs(message_segment))

                st.write("**Segmento de 5 segundos para análisis:**")
                st.audio(message, sample_rate=fs)

        except Exception as e:
            st.error(f"Error al procesar el archivo: {e}")

# --- Controles de Modulación ---
fc = st.sidebar.slider("2. Frecuencia de la portadora (fc) [Hz]", 4000, 15000, 5000, 100)
sideband_type = st.sidebar.radio(
    "3. Tipo de Banda Lateral:",
    ('Banda Lateral Superior (USB)', 'Banda Lateral Inferior (LSB)')
)

# --- Análisis ---
if message is not None:
    st.markdown("---")
    t = np.linspace(0, len(message)/fs, len(message), endpoint=False)

    st.header("Análisis de la Señal")
    with st.expander("Etapa 1: Señal Mensaje Original", expanded=True):
        st.markdown(r'''
        La señal mensaje, $m(t)$, es la información original que se desea transmitir. Su espectro, obtenido mediante la Transformada de Fourier, $M(f)$, nos muestra su contenido frecuencial. Para una transmisión eficiente, es crucial conocer el ancho de banda de esta señal.
        ''')
        plot_signal(t, message, "Señal Mensaje en el Tiempo")
        spectrum_xlim = 10 if msg_type == 'Pulso Rectangular' else 4000
        plot_spectrum(message, fs, "Espectro de la Señal Mensaje", x_lim=spectrum_xlim)

    st.header("Proceso de Modulación (Método de Hilbert)")
    with st.expander(f"Etapa 2: Señal Modulada {sideband_type}"):
        st.markdown(r'''
        Para generar una señal de Banda Lateral Única (SSB), se utiliza el método de la transformada de Hilbert. Este método desplaza en fase la señal mensaje en $-90^\circ$ para crear $\hat{m}(t)$. Luego, se combina con la señal original y portadoras en cuadratura.
        ''')
        if sideband_type == 'Banda Lateral Superior (USB)':
            st.latex(r"s_{USB}(t) = m(t) \cos(2\pi f_c t) - \hat{m}(t) \sin(2\pi f_c t)")
            m_hat_t = np.imag(signal.hilbert(message))
            ssb_signal = message * np.cos(2*np.pi*fc*t) - m_hat_t * np.sin(2*np.pi*fc*t)
        else: # LSB
            st.latex(r"s_{LSB}(t) = m(t) \cos(2\pi f_c t) + \hat{m}(t) \sin(2\pi f_c t)")
            m_hat_t = np.imag(signal.hilbert(message))
            ssb_signal = message * np.cos(2*np.pi*fc*t) + m_hat_t * np.sin(2*np.pi*fc*t)

        st.markdown(r'''
        El espectro resultante, $S(f)$, contiene únicamente la banda lateral seleccionada (superior o inferior) de la señal modulada en doble banda lateral, ahorrando así ancho de banda.
        ''')
        plot_signal(t, ssb_signal, f"Señal {sideband_type} en el Tiempo", y_max=0.6)
        plot_spectrum(ssb_signal, fs, f"Espectro de la Señal {sideband_type}", x_lim=fc+4000)

    st.header("Proceso de Demodulación")
    with st.expander("Etapa 3: Demodulación Coherente"):
        st.markdown(r'''
        La demodulación se realiza multiplicando la señal SSB recibida, $s(t)$, por una portadora local coherente, $c(t) = \cos(2\pi f_c t)$. Esto traslada el espectro de la señal de vuelta a la banda base (centrado en $0$ Hz) y también a una componente de alta frecuencia centrada en $2f_c$.
        $$v(t) = s(t) \cdot \cos(2\pi f_c t)$$
        ''')
        demod_mult = ssb_signal * np.cos(2 * np.pi * fc * t)
        plot_spectrum(demod_mult, fs, "Espectro post-multiplicación", x_lim=2*fc + 4000)

    with st.expander("Etapa 4: Filtro Pasa-Bajas (LPF)"):
        st.markdown(r'''
        Se diseña un filtro pasa-bajas (IIR tipo Butterworth) para eliminar la componente de alta frecuencia ($2f_c$) y recuperar únicamente la señal mensaje en banda base. A continuación se muestra la respuesta en frecuencia y el diagrama de polos y ceros del filtro.
        ''')
        lpf_cutoff = 3500 #Debe ser menor a fc
        nyquist = 0.5 * fs
        b_lpf, a_lpf = signal.butter(6, lpf_cutoff/nyquist, btype='low')
        plot_filter(b_lpf, a_lpf, fs, "Filtro Pasa-Bajas de Recuperación")

    with st.expander("Etapa 5: Señal Mensaje Recuperada", expanded=True):
        st.markdown(r'''
        Finalmente, se aplica el filtro pasa-bajas a la señal del paso anterior. La salida del filtro es la señal mensaje recuperada, que idealmente debería ser una réplica (escalada) de la señal original.
        ''')
        recovered_signal = signal.lfilter(b_lpf, a_lpf, demod_mult)
        fig, ax = plt.subplots(figsize=(10, 4))
        ax.plot(t, message, label="Original", alpha=0.7)
        ax.plot(t, recovered_signal, label="Recuperada", linestyle='--')
        ax.set_title("Comparación: Original vs. Recuperada")
        ax.legend()
        st.pyplot(fig)

        st.write("Escucha la señal recuperada:")
        wav_buffer = io.BytesIO()
        normalized_signal = (recovered_signal / np.max(np.abs(recovered_signal)) * 32767).astype(np.int16)
        write(wav_buffer, fs, normalized_signal)
        st.audio(wav_buffer, format='audio/wav')
else:
    st.info("Seleccione una fuente de señal para comenzar.")


Overwriting pages/2_📡_Modulación_SSB-AM.py


In [38]:
import os
import subprocess
import time
import re
import urllib.request

# ==============================================================================
# PASO 3: Descargar y configurar Cloudflared
# ==============================================================================
print("\n--- Configurando Cloudflare Tunnel ---")
try:
    urllib.request.urlretrieve("https://github.com/cloudflare/cloudflared/releases/latest/download/cloudflared-linux-amd64", "cloudflared")
    os.chmod("cloudflared", 0o755)
    # Usamos subprocess para mover el archivo y manejar errores
    subprocess.run(["mv", "cloudflared", "/usr/local/bin/cloudflared"], check=True)
except Exception as e:
    print(f"Error configurando Cloudflared: {e}")


# ==============================================================================
# PASO 4: Ejecutar Streamlit y el túnel de Cloudflare
# ==============================================================================
print("\n--- Ejecutando la aplicación Streamlit y el túnel ---")
# Ejecutar streamlit y guardar los logs
streamlit_process = subprocess.Popen(["streamlit", "run", "0_🏠_Inicio.py", "--server.port", "8501"], stdout=subprocess.PIPE, stderr=subprocess.PIPE)

# Ejecutar cloudflared y guardar los logs
with open("cloudflared.log", "w") as log_file:
    cloudflared_process = subprocess.Popen(["cloudflared", "tunnel", "--url", "http://localhost:8501"], stdout=log_file, stderr=subprocess.STDOUT)

time.sleep(5)  # Esperar que se genere la URL


# ==============================================================================
# PASO 5: Leer la URL pública y mantener la ejecución
# ==============================================================================
public_url = ""
try:
    with open('cloudflared.log', 'r') as f:
        for line in f:
            match = re.search(r'https?://\S+\.trycloudflare\.com', line)
            if match:
                public_url = match.group(0)
                print(f"\n✅ Tu aplicación está disponible en: {public_url}")
                break
except FileNotFoundError:
    print("\n❌ No se encontró el archivo de log de Cloudflare.")

if not public_url:
    print("\n❌ No se pudo obtener la URL de Cloudflare. Revisa los logs.")


print("\n========================================================================")
print("La aplicación Streamlit se está ejecutando en segundo plano.")
print("Para detener la ejecución, interrumpe la ejecución de esta celda (Ctrl+C).")
print("========================================================================")

try:
    while True:
        time.sleep(1)
except KeyboardInterrupt:
    print("\n--- Deteniendo la ejecución ---")
    streamlit_process.terminate()
    cloudflared_process.terminate()
    print("✅ Procesos de Streamlit y Cloudflared finalizados.")



--- Configurando Cloudflare Tunnel ---

--- Ejecutando la aplicación Streamlit y el túnel ---

✅ Tu aplicación está disponible en: https://throw-planner-webshots-exclusively.trycloudflare.com

La aplicación Streamlit se está ejecutando en segundo plano.
Para detener la ejecución, interrumpe la ejecución de esta celda (Ctrl+C).

--- Deteniendo la ejecución ---
✅ Procesos de Streamlit y Cloudflared finalizados.
